In [ ]:
from neo4j import GraphDatabase
from tkinter import messagebox, ttk
from tkinter import *
import tkinter
import matplotlib.pyplot as plt
import numpy as np
import csv
import random

#SE GENERA LA TABLA

tabla = (["","Joven","Adulto","Mayor"],["Bajo","Medio","Bajo","Medio"],["Medio","Alto","Medio","Alto"],["Alto","Alto","Alto","Alto"])
for t in tabla:
    print("|",t[0],"|",t[1],"|",t[2],"|",t[3])

In [ ]:
#CLASE PAR CREAR NODO CENTRAR-PARQUE CENTRAL
class CLASE_NEO4J(object): 
    def __init__(self):
        self._driver = GraphDatabase.driver("bolt:neo4j://localhost:7687", auth=("neo4j", "cuenca"), encrypted=False)
    def close(self):
        self._driver.close()
    def LISTAR(self):
        with self._driver.session() as session:
            greeting = session.write_transaction(self._LISTAR_PERSONAS)
    
    def KNN(self):
        with self._driver.session() as session:
            greeting = session.write_transaction(self._EJECUTAR_KNN) 
    def VER_PELICULA(self,persona,pelicula):
        with self._driver.session() as session:
            greeting = session.write_transaction(self._VER,persona,pelicula)
            
    def VISTAS(self,persona):
        with self._driver.session() as session:
            greeting = session.write_transaction(self._LISTAR_VISTAS,persona)
            
    def CREAR_PELICULA(self, message, nombre, rating):
        with self._driver.session() as session:
            greeting = session.write_transaction(self._VALIDAR_PELICULA, message, nombre, rating)
            print(greeting)
    def CREAR_PERSONA(self, message, nombre, edad, por_manejo, pel_vista, pel_rating):
        with self._driver.session() as session:
            greeting = session.write_transaction(self._VALIDAR_PERSONA, message, nombre, edad, por_manejo, pel_vista, pel_rating)
            print(greeting)

    #METODO PARA CREAR LOS NODOS DE LUGARES
    @staticmethod
    def _VALIDAR_PELICULA(tx, message, nombre, rating):
        #SE BUSCA SI EL LUGAR DEL ARREGLO EXISTE EN LA BASE NEO4J
        result2 = tx.run("match(l:Pelicula {nombre:'"+nombre+"'}) return l.nombre").data()
        #CONDICION PARA VERIFICAR SI EXISTE
        if int(len(result2)) == 0:
            print("SE CREA LA PELICULA EN LA BASE..........")
            #SE CREA NODO LUGAR
            result = tx.run("CREATE("+nombre+":Pelicula {nombre:'"+nombre+"' ,rating: "+rating+"}) "
                        "SET "+nombre+".message = $message "
                        "RETURN "+nombre+".message + ', from node ' + id("+nombre+")", message=message)
        elif int(len(result2))  == 1:
            print("EL NODO PELICULA YA EXISTE, INGRESAR OTRA PELICULA........")
    @staticmethod
    def _VALIDAR_PERSONA(tx, message,nombre, edad, por_manejo, pel_vista, pel_rating):
        #SE BUSCA SI EL LUGAR DEL ARREGLO EXISTE EN LA BASE NEO4J
        result2 = tx.run("match(l:Personas {nombre:'"+nombre+"'}) return l.nombre").data()
        #CONDICION PARA VERIFICAR SI EXISTE
        if int(len(result2)) == 0:
            print("SE CREA LA PERSONA EN LA BASE..........")
            #SE CREA NODO LUGAR
            result = tx.run("CREATE("+nombre+":Personas {nombre:'"+nombre+"' ,edad: "+edad+", manejo: "+por_manejo+", pelicula_vista: '"+pel_vista+"',rating: '"+pel_rating+"'}) "
                        "SET "+nombre+".message = $message "
                        "RETURN "+nombre+".message + ', from node ' + id("+nombre+")", message=message)
        elif int(len(result2))  == 1:
            print("EL NODO PERSONA YA EXISTE, INGRESAR OTRO PERSONA........")
    @staticmethod
    def _VER(tx,persona,pelicula):
        result2 = tx.run("match("+str(persona)+":Personas {nombre:'"+str(persona)+"'}) match("+str(pelicula)+":Pelicula {nombre:'"+str(pelicula)+"'}) create("+str(persona)+")-[:VER_PELICULA]->("+str(pelicula)+")")
        print("valor resultados es  >",result2)
        print("SE GENERA LA RELACION VER PELICULA",persona,pelicula)
        
        
    @staticmethod
    def _LISTAR_PERSONAS(tx):
        result = tx.run("match (p:Personas) return p.nombre as nombre,p.edad as edad,p.manejo as manejo").data()
        lista =[]
        for io in result:
            var = str(io.get("nombre"))+"   :   "+str(io.get("edad"))+"   :   "+str(io.get("manejo"))
            lista.append(var)
        combo["values"]=lista
    @staticmethod
    def _EJECUTAR_KNN(tx):
        #SE GENERA EL GRAFO CON LOS NODOS PARA EL ALGORITMO DE KNN
        result1 = tx.run("CALL gds.graph.create('graficoknn18',{Pelicula: {label: 'Pelicula',properties: 'rating'}},'*')")
        print("Resultado 1"+str(result1))
        #SE EJECUTA ALGORITMO KNN
        result = tx.run("CALL gds.beta.knn.stream('graficoknn18', {    topK: 1,    nodeWeightProperty: 'rating',    randomSeed: 42,    concurrency: 1,    sampleRate: 1.0,    deltaThreshold: 0.0}) YIELD node1, node2, similarity RETURN gds.util.asNode(node1).nombre AS Pelicula1, gds.util.asNode(node2).nombre AS Pelicula2, gds.util.asNode(node2).rating AS Rating, similarity ORDER BY similarity DESCENDING, Pelicula1, Pelicula2").data()
        lista =[]
        for io in result:
            var = str(io.get("Pelicula1"))+" : "+str(io.get("Pelicula2"))+" : "+str(io.get("Rating"))
            lista.append(var)
        combo2["values"]=lista
    @staticmethod
    def _LISTAR_VISTAS(tx,persona):
        result = tx.run("match(p:Personas) -[:VER_PELICULA]->(pe:Pelicula) where p.nombre ='"+persona+"' return pe.nombre").data()
        lista =[]
        print(result)
        for io in result:
            var = str(io.get("pe.nombre"))
            lista.append(var)
        combo3["values"]=lista
            
#SE INICIALIZA LA CLASE DE LOS METODOS DE NEO4J
grafo=CLASE_NEO4J()

In [ ]:
ll= []
with open('peliculas.csv', newline='') as File:  
    reader = csv.reader(File)
    for row in reader:
        #print(row)
        ll.append(row)
        print(row[0])
        print(row[1])
        print(row[2])
        grafo.CREAR_PELICULA("SE GENERA UN NODO PELICULA EN LA BASE >>>>>>>>> ",str(row[1]), str(float(row[2])))

def randomp():
    i = random.randint(0, 19)
    p = ll[i][1]
    r = ll[i][2]
    return p, r

def edad():
    i = random.randint(18, 70)
    return i
def manejo():
    por_manejo = random.randint(0, 100)
    return por_manejo
p = "Persona"
for i in range(1, 101):
    pp = "{}_{}".format(p, i)
    grafo.CREAR_PERSONA("SE GENERA UN NODO PERSONA EN LA BASE >>>>>>>>> ",str(pp), str(edad()), str(manejo()), str(randomp()[0]), str(float(randomp()[1])))

In [ ]:
import tkinter as tk
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg


manejo=0.0
edad=0
resultados =([])
resultadosm =([])
listax1 = [18,19,20,21,22,23,24,25,26,27,28,29,30,20,23,26,29,32,35,38,41,44,47,50,40,44,48,52,56,60,61,62,63,64,65,66,67,68,69,70]
listay1 = [1,1,1,1,1,1,1,1,0.8,0.6,0.4,0.2,0,0,0.2,0.4,0.6,0.8,1,0.8,0.6,0.4,0.2,0,0,0.2,0.4,0.6,0.8,1,1,1,1,1,1,1,1,1,1,1]
listax = [0,10,11,13,15,17,19,21,10,16,22,28,34,40,44,48,52,56,60,50,54,58,62,66,70,80,90,100]
listay = [1,1,1,0.8,0.6,0.4,0.2,0,0,0.2,0.4,0.6,0.8,1,0.8,0.6,0.4,0.2,0,0,0.2,0.4,0.6,0.8,1,1,1,1]
listax2 = [0,10,12,14,16,18,20,10,14,18,22,26,30,33,36,39,42,45,40,43,46,49,52,55,60,70,80,90,100]
listay2 = [1,1,0.8,0.6,0.4,0.2,0,0,0.2,0.4,0.6,0.8,1,0.8,0.6,0.4,0.2,0,0,0.2,0.4,0.6,0.8,1,1,1,1,1,1]

def listar_pelis():
    st = str(combo.get()).split(" : ")
    #PERSONA
    print("VALOR DE PERSONAS>> ",str(st[0]))
    per = str(st[0])    
    grafo.VISTAS(str(per).replace(" ",""))
    tkinter.messagebox.showinfo(title="SE PRESIONA", message="Se Listan las peliculas vistas")

    
def listarp():
    pv = grafo.LISTAR()
    tkinter.messagebox.showinfo(title="SE PRESIONA", message="Se listaron con exitos")

def ver_peli():    
    st = str(combo.get()).split(" : ")
    #PERSONA
    per = str(st[0])    
    st2 = str(peli.get())
    #PELICULA
    pel = st2    
    grafo.VER_PELICULA(str(per).replace(" ",""),str(pel).replace(" ",""))
    tkinter.messagebox.showinfo(title="SE PRESIONA", message="Esta viendo la pelicula Ingresada")

def logicadif():  
    st = str(combo.get()).split("   :   ")
    print("<-----SE EJECUTA LA LOGICA DIFUSA-----> ")
    ax1.axvline(int(st[1]), label='pyplot vertical line',color='red')
    line.draw()
    ax.axvline(int(st[2]), label='pyplot vertical line',color='blue')
    line.draw()
    #VARIABLE DE EDAD
    a = int(st[1])
    #VARIABLE DE MANEJO
    b = int(st[2])    
    cont=0    
    for lx in listax1[:13]:     
        if int(lx) == int(a):
            print("*********JOVEN****************",lx,listay1[cont])
            resultados.insert(0,["Medio","Alto","Alto",lx])
        elif int(lx) != int(a):
            pass
        cont =cont +1   
    #SE BUSCA EL VALOR INGRESADO EN EL VECTOR DE X ESTA EN T MEDIA
    cont1=0
    lisyn = listay[13:33]
    for lx in listax[13:33]:  
        if int(lx) == int(a):
            print("***********ADULTO********",lx,lisyn[cont1])
            resultados.insert(1,["Bajo","Medio","Alto",lx])
        elif int(lx) != int(a):
            pass
        cont1 = cont1 +1   
    #SE BUSCA EL VALOR INGRESADO EN EL VECTOR DE X ESTA T ALTA
    cont=0
    lista = listay[34:]
    for lx in listax[34:]:        
        if int(lx) == int(a):
            print("**********MAYOR*********",lx,lista[cont])
            resultados.insert(2,["Medio","Alto","Alto",lx])
        elif int(lx) != int(a):
            pass
        cont =cont +1    
    #SE VALIDA  LOS VALORES DE MANEJO
    cont2=0
    lisyn4 = listay[19:]
    for lx in listax[19:]:  
        if int(lx) == int(b):
            print("**********PORCENTAJE ALTO*********",lx,lisyn4[cont2])
            resultadosm.insert(0,["Alto","Alto","Alto",lisyn4[cont2]])
        elif int(lx) != int(b):
            pass
        cont2 = cont2 +1   
    #SE BUSCA EL VALOR INGRESADO EN EL VECTOR DE X ESTA T ALTA
    cont23=0
    lista5 = listay[8:19]
    for lx in listax[8:19]:        
        if int(lx) == int(b):
            print("************PORCENTAJE MEDIO*********",lx,lista5[cont23])
            resultadosm.insert(1,["Alto","Medio","Alto",lista5[cont23]])
        elif int(lx) != int(b):
            pass
        cont23 =cont23 +1 
    #PORCNTAJE BAJO
    cont24=0
    lista6 = listay[:8]
    for lx in listax[:8]:        
        if int(lx) == int(b):
            print("*********PORCENTAJE BAJO*******",lx,lista6[cont24])
            resultadosm.insert(2,["Alto","Medio","Alto",lista6[cont24]])
        elif int(lx) != int(b):
            pass
        cont24 =cont24 +1 
    print("VALOR DE LOS RESULTADOS DE EDADES >> ",resultados) 
    print("VALOR DE LOS MANEJOS>> ",resultadosm)
    ct=0
    for rs in resultados:
        if str(rs)!="[]":
            print("EDAD------->",rs)        
        if str(resultadosm)!="[]":
            print("MANEJO------->",resultadosm[ct])
        ct=ct+1
    #SE REALIZA LA RECOMENDACIONES BASADAS EN LAS SIMILITUDES
    grafo.KNN()
    tkinter.messagebox.showinfo(title="SE EJECUTARA KNN ", message="Selecione una de las 2 peliculas a ver")

def B0f():    
    ax1.clear()
    ax1.plot(listax1[:13], listay1[:13],label="Joven")
    ax1.legend()
    ax1.plot(listax1[13:24], listay1[13:24],label="Adulto")    
    ax1.legend()   
    ax1.plot(listax1[24:], listay1[24:], label="Mayor")     
    ax1.legend()    
    ax1.grid(True)
    ax1.set_xlabel('$x$'),ax1.set_ylabel('$y$')
    ax1.set_title('EDADES')
    line.draw()
    
   
    ax.clear()
    ax.plot(listax[:8], listay[:8],label="Bajo")
    ax.legend()
    ax.plot(listax[8:19], listay[8:19],label="Medio")    
    ax.legend()   
    ax.plot(listax[19:], listay[19:], label="Alto")     
    ax.legend()    
#     ax.fill_between(listax[:21],listay[:21],color="green")
    ax.grid(True)
    ax.set_xlabel('$x$'),ax.set_ylabel('$y$')
    ax.set_title('% MANEJO')
    line.draw() 
    
    ax2.clear()
    ax2.plot(listax2[:7], listay2[:7],label="Bajo")
    ax2.legend()
    ax2.plot(listax2[7:18], listay2[7:18],label="Medio")    
    ax2.legend()   
    ax2.plot(listax2[18:], listay2[18:], label="Alto")     
    ax2.legend()    
#     ax.fill_between(listax[:21],listay[:21],color="green")
    ax2.grid(True)
    ax2.set_xlabel('$x$'),ax2.set_ylabel('$y$')
    ax2.set_title('% RIESGO FINANCIERO')
    line.draw()
    
#--- Raiz ---
root = tk.Tk()
root.geometry('940x450')
root.title("Tkinter + Matplotlib")
#------------

#-- Frames ---
left_frame = tk.Frame(root)
left_frame.place(relx=0.03, rely=0.05, relwidth=0.25, relheight=0.9)

right_frame = tk.Frame(root, bg='#C0C0C0', bd=1.5)
right_frame.place(relx=0.3, rely=0.05, relwidth=0.65, relheight=0.9)
#---------------
#LABEL--------------

label = Label(left_frame, text="LISTADO DE PERSONAS", relief=RAISED )
label.place(relx=0.05, rely=0.24,relheight=0.045, relwidth=0.95)

label2 = Label(left_frame, text="Pelicula1 : Pelicula2 : Rating", relief=RAISED )
label2.place(relx=0.05, rely=0.46,relheight=0.045, relwidth=0.95) 
#-------------
#--- Botones ---

B0 = tk.Button(left_frame,text="PRESENTAR GRAFICOS", foreground = "white",background = "brown",command = B0f)
B0.place(relx=0.05, rely=0.08,relheight=0.06, relwidth=1)

B1 =tk.Button(left_frame,text="LISTAR PERSONAS",foreground = "white",background = "brown",command = listarp)
B1.place(relx=0.05, rely=0.16,relheight=0.06, relwidth=1)

B2 =tk.Button(left_frame,text="VALIDAR RIESGO PERSONA ", foreground = "white",background = "green",command = logicadif)
B2.place(relx=0.05, rely=0.38,relheight=0.06, relwidth=1)

B3 =tk.Button(left_frame,text="VER PELICULA ", foreground = "white",background = "brown",command = ver_peli)
B3.place(relx=0.05, rely=0.70,relheight=0.06, relwidth=1)

B4 =tk.Button(left_frame,text="PELICULAS VISTAS ", foreground = "white",background = "brown",command = listar_pelis)
B4.place(relx=0.05, rely=0.82,relheight=0.06, relwidth=1)

#----COMBOBOX---------
combo = ttk.Combobox(left_frame, state="readonly")
combo.place(relx=0.05, rely=0.30,relheight=0.05, relwidth=0.95)

combo2 = ttk.Combobox(left_frame, state="readonly")
combo2.place(relx=0.05, rely=0.54,relheight=0.05, relwidth=0.95)

combo3 = ttk.Combobox(left_frame, state="readonly")
combo3.place(relx=0.05, rely=0.90,relheight=0.05, relwidth=0.95)
#-------entry
peli = Entry(left_frame, bd =5)
peli.place(relx=0.05, rely=0.62,relheight=0.07, relwidth=0.95)
#------------
#--- Agregar figura ---
figure = plt.Figure(figsize=(5,6), dpi=100)
ax = figure.add_subplot(322)
ax.grid(True),ax.set_xlabel('$x$'),ax.set_ylabel('$y(x)$')

ax1 = figure.add_subplot(321)
ax1.grid(True),ax1.set_xlabel('$x$'),ax1.set_ylabel('$y(x)$')


ax2 = figure.add_subplot(212)
ax2.grid(True),ax1.set_xlabel('$x$'),ax1.set_ylabel('$y(x)$')

line = FigureCanvasTkAgg(figure, right_frame)
line.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH,expand=1)
#----------------------

root.mainloop()